# God Pipeline

In [152]:
import sys
import os
from dotenv import load_dotenv
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

#1. load environment variables and data

# load environment variables
load_dotenv()

#add working directory to sys path to execute utils/dataset.py
working_dir = os.environ.get("WORKING_DIRECTORY")
sys.path.insert(0, working_dir)
from utils.dataset import get_data 
df = get_data()
df.head(10)

Loading data from wines: 0it [00:00, ?it/s]Exception during reset or similar
Traceback (most recent call last):
  File "/opt/anaconda3/envs/data-science/lib/python3.11/site-packages/sqlalchemy/pool/base.py", line 991, in _finalize_fairy
    fairy._reset(
  File "/opt/anaconda3/envs/data-science/lib/python3.11/site-packages/sqlalchemy/pool/base.py", line 1440, in _reset
    pool._dialect.do_rollback(self)
  File "/opt/anaconda3/envs/data-science/lib/python3.11/site-packages/sqlalchemy/engine/default.py", line 657, in do_rollback
    dbapi_connection.rollback()
MySQLdb.OperationalError: (2013, 'Lost connection to MySQL server during query')
Exception during reset or similar
Traceback (most recent call last):
  File "/opt/anaconda3/envs/data-science/lib/python3.11/site-packages/sqlalchemy/pool/base.py", line 991, in _finalize_fairy
    fairy._reset(
  File "/opt/anaconda3/envs/data-science/lib/python3.11/site-packages/sqlalchemy/pool/base.py", line 1440, in _reset
    pool._dialect.do_rol

,wine type,fixed acidity,volatile acidity,citric acid,residual sugar,magnesium,flavanoids,minerals,calcium,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,Pinot noir,5.8,0.15,0.49,1.1,76.729301,894.94,186.639301,109.91,0.048,21.0,98.0,0.99290,3.19,0.48,9.2,5
1,Merlot,6.6,0.25,0.32,5.6,4.795712,1160.95,251.875712,247.08,0.039,15.0,68.0,0.99163,2.96,0.52,11.1,6
2,Chardonnay,6.7,0.21,0.34,1.5,85.193710,789.82,304.703710,219.51,0.035,45.0,123.0,0.98949,3.24,0.36,12.6,7
3,Merlot,8.3,0.28,0.27,17.5,11.976525,777.86,237.586525,225.61,0.045,48.0,253.0,1.00014,3.02,0.56,9.1,6
4,Merlot,7.5,0.42,0.19,6.9,5.599673,785.72,95.399673,89.80,0.041,62.0,150.0,0.99508,3.23,0.37,10.0,6
5,Merlot,7.3,0.34,0.30,1.3,22.403749,1044.95,289.523749,267.12,0.057,25.0,173.0,0.99480,3.26,0.51,9.1,6
6,Merlot,7.6,0.21,0.49,2.5,23.875866,888.61,133.545866,109.67,0.047,20.0,130.0,0.99178,3.15,0.48,11.1,5
7,Chardonnay,6.0,0.25,0.40,5.7,23.309699,1381.79,266.529699,243.22,0.052,56.0,152.0,0.99398,3.16,0.88,10.5,6
8,Cabernet Sauvignon,6.7,0.18,0.19,4.7,49.165745,1456.41,269.915745,220.75,0.046,57.0,161.0,0.99460,3.32,0.66,10.5,6
9,Gamay,7.7,0.28,0.39,8.9,54.450579,929.44,377.690579,323.24,0.036,8.0,117.0,0.99350,3.06,0.38,12.0,2


In [153]:
categorical_features = df.select_dtypes(include=['object']).columns
numerical_features = df.select_dtypes(include=[np.number]).columns
#drop 'quality' from numerical features (its a series)
numerical_features = numerical_features.drop('quality')
label = pd.Series('quality')

## God Pipeline

In [223]:
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import FunctionTransformer
from sklearn.preprocessing import OneHotEncoder
#import FunctionTransformer

### Cleaning Pipeline

In [224]:
def outlier_detection_label(df):
    #detect outliers and impute them with the simple imputer
    from sklearn.impute import KNNImputer

    #detect outliers with z-score and set them to NaN with = np.nan
    from scipy import stats
    z = np.abs(stats.zscore(df.iloc[:, df.shape[1]-1]))
    df.iloc[:, df.shape[1]-1][(z >= 3)] = np.nan


    #impute outliers with linear regression
    imputer = KNNImputer(n_neighbors=5).set_output(transform="pandas")
    df = imputer.fit_transform(df)
    
    return df

def outlier_detection(df):
    #detect outliers and impute them with the simple imputer
    from sklearn.impute import SimpleImputer

    #detect outliers
    Q1 = df.quantile(0.25)
    Q3 = df.quantile(0.75)
    IQR = Q3 - Q1

    #detect outliers for each column and set them to NaN
    for col in df.columns:
        df.loc[(df[col] < (Q1[col] - 1.5 * IQR[col])) | (df[col] > (Q3[col] + 1.5 * IQR[col])), col] = np.nan

    #impute outliers with median
    imputer = SimpleImputer(strategy='median').set_output(transform="pandas")
    df = imputer.fit_transform(df)

    return df
outlier_detection_label = FunctionTransformer(outlier_detection_label).set_output(transform="pandas")
outlier_detection = FunctionTransformer(outlier_detection).set_output(transform="pandas")


/opt/anaconda3/envs/data-science/lib/python3.11/site-packages/sklearn/preprocessing/_function_transformer.py:343: UserWarning: With transform="pandas", `func` should return a DataFrame to follow the set_output API.
  warnings.warn(


In [225]:
cleaning_pipeline = Pipeline(steps=[
])

categorical_imputer = SimpleImputer(strategy="most_frequent").set_output(transform="pandas")

numerical_imputer = SimpleImputer(strategy="mean").set_output(transform="pandas")

#pipeline for categorical features
categorical_pipeline = Pipeline(steps=[])
categorical_pipeline.steps.append(('imputer', categorical_imputer))
categorical_pipeline.steps.append(('onehot', OneHotEncoder(handle_unknown='ignore', sparse_output=False).set_output(transform="pandas")))

#pipeline for numerical features
numeric_pipeline = Pipeline(steps=[])
numeric_pipeline.steps.append(('imputer', numerical_imputer))
numeric_pipeline.steps.append(('outlier_detection', outlier_detection))

#pipeline for label
label_pipeline = Pipeline(steps=[])
label_pipeline.steps.append(('imputer', numerical_imputer))

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_pipeline, numerical_features),
        ('cat', categorical_pipeline, categorical_features),
        ('label', label_pipeline, label)
    ]).set_output(transform="pandas")
cleaning_pipeline.steps.append(('preprocessor', preprocessor))
cleaning_pipeline.steps.append(("outlier_detection_label", outlier_detection_label))
cleaning_pipeline

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer()),
                                                                  ('outlier_detection',
                                                                   FunctionTransformer(func=<function outlier_detection at 0x152e0a160>))]),
                                                  Index(['fixed acidity', 'volatile acidity', 'citric acid', 'residual sugar',
       'magnesium', 'flavanoids', 'minerals', 'calcium', 'chlorides',
       'free sulfur dioxid...
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('onehot',
                                                                   OneHotEncoder(handle_unknown='ignore',
                                                                                 sparse_output=False))]),
                                                  Index(['wine type'], dtype='object')),
                                                 ('label',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer())]),
                                                  0    quality
dtype: object)])),
                ('outlier_detection_label',
                 FunctionTransformer(func=<function outlier_detection_label at 0x152e0b060>))])

In [226]:
from sklearn.linear_model import LinearRegression

training_pipeline_linear_regression = Pipeline(steps=[
    ('linear_regression', LinearRegression())
])
model = {
        "name": "LinearRegression",
        "estimator": LinearRegression(),
        "hyperparameters":
            {
                "model__fit_intercept": [True, False],
                "model__copy_X": [True, False],
                "model__n_jobs": [1,5,10,20],
            }
    }

In [227]:
#God Function based on example: LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV

def god_function(dirty_df):
    clean_df = pd.DataFrame(cleaning_pipeline.fit_transform(dirty_df))
    X = clean_df.drop('quality', axis=1)
    y = clean_df['quality']
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)
    
   
    return clean_df

pd.DataFrame(god_function(df))

,num__fixed acidity,num__volatile acidity,num__citric acid,num__residual sugar,num__magnesium,num__flavanoids,num__minerals,num__calcium,num__chlorides,num__free sulfur dioxide,...,num__density,num__pH,num__sulphates,num__alcohol,cat__wine type_Cabernet Sauvignon,cat__wine type_Chardonnay,cat__wine type_Gamay,cat__wine type_Merlot,cat__wine type_Pinot noir,label__quality
0,5.8,0.15,0.49,1.1,76.729301,894.94,186.639301,109.91,0.048,21.0,...,0.99290,3.19,0.48,9.2,0.0,0.0,0.0,0.0,1.0,5.0
1,6.6,0.25,0.32,5.6,4.795712,1160.95,251.875712,247.08,0.039,15.0,...,0.99163,2.96,0.52,11.1,0.0,0.0,0.0,1.0,0.0,6.0
2,6.7,0.21,0.34,1.5,85.193710,789.82,304.703710,219.51,0.035,45.0,...,0.98949,3.24,0.36,12.6,0.0,1.0,0.0,0.0,0.0,7.0
3,8.3,0.28,0.27,17.5,11.976525,777.86,237.586525,225.61,0.045,48.0,...,1.00014,3.02,0.56,9.1,0.0,0.0,0.0,1.0,0.0,6.0
4,7.5,0.42,0.19,6.9,5.599673,785.72,95.399673,89.80,0.041,62.0,...,0.99508,3.23,0.37,10.0,0.0,0.0,0.0,1.0,0.0,6.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7995,6.4,0.23,0.37,7.9,92.701914,1143.32,318.791914,226.09,0.050,60.0,...,0.99488,2.86,0.49,9.3,1.0,0.0,0.0,0.0,0.0,6.0
7996,7.0,0.22,0.26,9.2,94.807955,863.32,322.107955,227.30,0.027,37.0,...,0.99228,3.06,0.34,12.5,1.0,0.0,0.0,0.0,0.0,8.0
7997,7.5,0.26,0.30,4.6,50.112474,831.67,360.872474,310.76,0.027,29.0,...,0.99085,3.15,0.38,12.0,0.0,0.0,0.0,1.0,0.0,7.0
7998,6.3,0.43,0.32,8.8,84.805688,1300.32,328.915688,244.11,0.042,18.0,...,0.99172,3.28,0.33,12.9,0.0,1.0,0.0,0.0,0.0,7.0
